In [1]:
size=256
overl=0.5
epochs=3
modelname="model3_256"
batchsize=10
training=True
base='/home/john/GIT/VipsML'

In [2]:

from keras.utils import Sequence
import pyvips as Vips
from random import shuffle, choice
from numpy import rot90, flip, frombuffer, uint8, asarray, squeeze
from math import ceil, floor
from functools import reduce

class SegNetImage(Sequence):    
    """Generator class to be fed to a keras fit-function."""

    def __init__(self,imagePath,maskpath=None,frame=256,overlay=0.0, batchSize=10):
        """Args:
            imagePath: Path to original image.
            maskpath: Path to image masks, None if not training.
            frame: Frame size (x and y dim)
            overlay: How much overlay to keep between each piece of the image.
            batchSize: Batchsize for feeding into CNN."""
        
        self.orig=Vips.Image.new_from_file(imagePath)        
        h,w,d = self.orig.width, self.orig.height, self.orig.bands        
        self.origShape = (w,h,d)
        self.frame=frame
        self.overlay=round(self.frame*overlay)
        self.step = self.frame-self.overlay        
        self.batchSize=batchSize                
        
        #The last step will cover a whole frame
        self.x_frames=ceil((w-(self.frame-self.step))/(self.step))
        self.y_frames=ceil((h-(self.frame-self.step))/(self.step))        
        self.total_frames=self.x_frames*self.y_frames
        
        # Fix padding 
        _w = ceil(self.x_frames*self.step)+(self.frame-self.step)
        _h = ceil(self.y_frames*self.step)+(self.frame-self.step)
        self.embeddedShape = (_w, _h, d)
        self.orig=self.orig.embed(0,0,_w,_h,extend='white')    
                
        if maskpath == None:
            self.training=False
        else:
            self.training=True
            self.initMask(maskpath)
            self.frame_order = list(range(self.total_frames))
            # The fit function will randomize the batch order,
            # but not the individual datasets inside each batch:
            shuffle(self.frame_order)
        
    def initMask(self,maskpath):
        self.mask=Vips.Image.new_from_file(maskpath)
        self.mask=self.mask.embed(0,0,self.embeddedShape[0],self.embeddedShape[1],extend='black')
        # We need a binary class matrix for training.
        # We could use keras.utils.to_categorical downstream instead.
        self.hist=self.mask.hist_find()
        self.featureN = self.hist.width
        features=list(range(self.featureN))
        expanded = None
        for f in features:
            if expanded:
                expanded=expanded.bandjoin(self.mask==f)
            else:
                expanded = self.mask==f
        self.mask=expanded  
        
    def __len__(self):
        return int(ceil(self.total_frames/self.batchSize))
    
    def __getitem__(self,index):
        # the keras.utils.Sequence expects one batch upon requesting an item
        start = index*self.batchSize
        end = min((index+1)*(self.batchSize),self.total_frames)
        if self.training:
            origs = []
            masks = []
            # We iterate one by one to assign random rotation and flipping
            # to both image and mask
            for i in self.frame_order[start:end]:
                rot=choice(range(4))
                flippage=choice(range(2))==1
                origs += [self.getSingleItem(i,rot,flippage)]
                # Output vector is expected to be 1D (could be fixed in the model definition):
                masks += [self.getMask(i,rot,flippage).reshape((self.frame*self.frame,self.featureN))]
            return asarray(origs),asarray(masks).astype(float)
        else:
            return asarray([self.getSingleItem(i) for i in range(start,end)])
    
    def getSingleItem(self,index,rot=0,flp=False):
        rotations=[lambda x: x,lambda x: x.rot90(), lambda x: x.rot180(), lambda x: x.rot270()]
        y = floor(index/self.x_frames)
        x = index % self.x_frames
        x_px = x*(self.step)
        y_px = y*(self.step)
        o=self.getImageAt(x_px, y_px, self.frame)
        o=rotations[rot](o)
        if flp:
            o=o.fliphor()
        return self.toNp(o)
        
    def getMask(self,index,rot=0,flp=False):
        rotations=[lambda x: x,lambda x: x.rot90(), lambda x: x.rot180(), lambda x: x.rot270()]
        y = floor(index/self.x_frames)
        x = index % self.x_frames
        x_px = x*(self.step)
        y_px = y*(self.step)
        o=self.getImageAt(x_px,y_px,self.frame,im=self.mask)
        o=rotations[rot](o)
        if flp:
            o=o.fliphor()
        return self.toNp(o)

    def getShape(self):
        return (self.frame, self.frame, 3)
        
    def toNp(self,im):
        return frombuffer(im.write_to_memory(), dtype=uint8).reshape(im.width, im.height, im.bands)
     
    def getBoxFrom(self,x,y,w):
        return (x,y,x+w,y+w)

    def getImageAt(self,x,y,w,im=None):
        if im is None:
          im=self.orig
        return im.crop(x,y,w,w)

    def setBatchSize(self,n):
        self.batchSize=n
    
    def on_epoch_end(self):
        # re-randomize upon finishing an epoch
        shuffle(self.frame_order)

class MultiSegNetImage(Sequence):
    def __init__(self,origs,masks=None,frame=256,overlay=0.0, batchSize=10):
        self.frame=frame
        self.overlay=overlay
        self.batchSize=batchSize
        self.training = masks!=None
        self.images = [SegNetImage(origs[i],masks[i] if self.training else None,frame,overlay,batchSize) for i in range(len(origs))]
        self.indices = reduce(lambda acc, val: acc+[(val,i) for i in range(self.images[val].total_frames)],range(len(self.images)),[])
        shuffle(self.indices)
    
    def getShape(self):
        """ Peek ahead at first image in stack"""
        return self.images[0].getShape()
    
    def totalItems(self):
        """ Total number of items """
        return len(self.indices)
        
    def __len__(self):
        """ Number of batches, not items """
        return int(ceil(self.totalItems()/self.batchSize))
    
    def __getitem__(self,index):
        """ One batch, not one item """
        start = index*self.batchSize
        end = min((index+1)*(self.batchSize),self.totalItems())
        if self.training:
            origs = []
            masks = []
            # We iterate one by one to assign (same) random rotation
            # and flipping to both image and mask
            for entry in self.indices[start:end]:
                imageN,itemN = entry
                image=self.images[imageN]
                rot=choice(range(4))
                flippage=choice(range(2))==1
                origs += [image.getSingleItem(itemN,rot,flippage)]
                # Output vector is expected to be 1D (could be fixed in the model definition):
                masks += [image.getMask(itemN,rot,flippage).reshape((image.frame*image.frame,image.featureN))]
            return asarray(origs),asarray(masks).astype(float)
        else:
            return asarray([self.images[i].getSingleItem(n) for i,n in indices[start:end]])
        
    def on_epoch_end(self):
        # re-randomize upon finishing an epoch
        shuffle(self.indices)
            

#K.set_floatx('float32')
#K.set_epsilon(1e-7) 

from keras import backend as K
from keras.layers import Layer



# Creds to ykamikawa/tf-keras-SegNet
class MaxPoolingWithArgmax2D(Layer):

    def __init__(
            self,
            pool_size=(2, 2),
            strides=(2, 2),
            padding='same',
            **kwargs):
        super(MaxPoolingWithArgmax2D, self).__init__(**kwargs)
        self.padding = padding
        self.pool_size = pool_size
        self.strides = strides

    def call(self, inputs, **kwargs):
        padding = self.padding
        pool_size = self.pool_size
        strides = self.strides
        if K.backend() == 'tensorflow':
            ksize = [1, pool_size[0], pool_size[1], 1]
            padding = padding.upper()
            strides = [1, strides[0], strides[1], 1]
            output, argmax = K.tf.nn.max_pool_with_argmax(
                    inputs,
                    ksize=ksize,
                    strides=strides,
                    padding=padding)
        else:
            errmsg = '{} backend is not supported for layer {}'.format(
                    K.backend(), type(self).__name__)
            raise NotImplementedError(errmsg)
        argmax = K.cast(argmax, K.floatx())
        return [output, argmax]

    def compute_output_shape(self, input_shape):
        ratio = (1, 2, 2, 1)
        output_shape = [
                dim//ratio[idx]
                if dim is not None else None
                for idx, dim in enumerate(input_shape)]
        output_shape = tuple(output_shape)
        return [output_shape, output_shape]

    def compute_mask(self, inputs, mask=None):
        return 2 * [None]


class MaxUnpooling2D(Layer):
    def __init__(self, size=(2, 2), **kwargs):
        super(MaxUnpooling2D, self).__init__(**kwargs)
        self.size = size

    def call(self, inputs, output_shape=None):
        updates, mask = inputs[0], inputs[1]
        with K.tf.variable_scope(self.name):
            mask = K.cast(mask, 'int32')
            input_shape = K.tf.shape(updates, out_type='int32')
            #  calculation new shape
            if output_shape is None:
                output_shape = (
                        input_shape[0],
                        input_shape[1]*self.size[0],
                        input_shape[2]*self.size[1],
                        input_shape[3])
            self.output_shape1 = output_shape

            # calculation indices for batch, height, width and feature maps
            one_like_mask = K.ones_like(mask, dtype='int32')
            batch_shape = K.concatenate(
                    [[input_shape[0]], [1], [1], [1]],
                    axis=0)
            batch_range = K.reshape(
                    K.tf.range(output_shape[0], dtype='int32'),
                    shape=batch_shape)
            b = one_like_mask * batch_range
            y = mask // (output_shape[2] * output_shape[3])
            x = (mask // output_shape[3]) % output_shape[2]
            feature_range = K.tf.range(output_shape[3], dtype='int32')
            f = one_like_mask * feature_range

            # transpose indices & reshape update values to one dimension
            updates_size = K.tf.size(updates)
            indices = K.transpose(K.reshape(
                K.stack([b, y, x, f]),
                [4, updates_size]))
            values = K.reshape(updates, [updates_size])
            ret = K.tf.scatter_nd(indices, values, output_shape)
            return ret

    def compute_output_shape(self, input_shape):
        mask_shape = input_shape[1]
        return (
                mask_shape[0],
                mask_shape[1]*self.size[0],
                mask_shape[2]*self.size[1],
                mask_shape[3]
                )

from keras.models import Model
from keras.layers import Input
from keras.layers.core import Activation, Reshape
from keras.layers.convolutional import Convolution2D
from keras.layers.normalization import BatchNormalization

#from layers import MaxPoolingWithArgmax2D, MaxUnpooling2D


def segnet(
        input_shape,
        n_labels,
        kernel=3,
        pool_size=(2, 2),
        output_mode="softmax"):
    # encoder
    inputs = Input(batch_shape=(None,)+input_shape)

    conv_1 = Convolution2D(64, (kernel, kernel), padding="same")(inputs)
    conv_1 = BatchNormalization()(conv_1)
    conv_1 = Activation("relu")(conv_1)
    conv_2 = Convolution2D(64, (kernel, kernel), padding="same")(conv_1)
    conv_2 = BatchNormalization()(conv_2)
    conv_2 = Activation("relu")(conv_2)

    pool_1, mask_1 = MaxPoolingWithArgmax2D(pool_size)(conv_2)

    conv_3 = Convolution2D(128, (kernel, kernel), padding="same")(pool_1)
    conv_3 = BatchNormalization()(conv_3)
    conv_3 = Activation("relu")(conv_3)
    conv_4 = Convolution2D(128, (kernel, kernel), padding="same")(conv_3)
    conv_4 = BatchNormalization()(conv_4)
    conv_4 = Activation("relu")(conv_4)

    pool_2, mask_2 = MaxPoolingWithArgmax2D(pool_size)(conv_4)

    conv_5 = Convolution2D(256, (kernel, kernel), padding="same")(pool_2)
    conv_5 = BatchNormalization()(conv_5)
    conv_5 = Activation("relu")(conv_5)
    conv_6 = Convolution2D(256, (kernel, kernel), padding="same")(conv_5)
    conv_6 = BatchNormalization()(conv_6)
    conv_6 = Activation("relu")(conv_6)
    conv_7 = Convolution2D(256, (kernel, kernel), padding="same")(conv_6)
    conv_7 = BatchNormalization()(conv_7)
    conv_7 = Activation("relu")(conv_7)

    pool_3, mask_3 = MaxPoolingWithArgmax2D(pool_size)(conv_7)

    conv_8 = Convolution2D(512, (kernel, kernel), padding="same")(pool_3)
    conv_8 = BatchNormalization()(conv_8)
    conv_8 = Activation("relu")(conv_8)
    conv_9 = Convolution2D(512, (kernel, kernel), padding="same")(conv_8)
    conv_9 = BatchNormalization()(conv_9)
    conv_9 = Activation("relu")(conv_9)
    conv_10 = Convolution2D(512, (kernel, kernel), padding="same")(conv_9)
    conv_10 = BatchNormalization()(conv_10)
    conv_10 = Activation("relu")(conv_10)

    pool_4, mask_4 = MaxPoolingWithArgmax2D(pool_size)(conv_10)

    conv_11 = Convolution2D(512, (kernel, kernel), padding="same")(pool_4)
    conv_11 = BatchNormalization()(conv_11)
    conv_11 = Activation("relu")(conv_11)
    conv_12 = Convolution2D(512, (kernel, kernel), padding="same")(conv_11)
    conv_12 = BatchNormalization()(conv_12)
    conv_12 = Activation("relu")(conv_12)
    conv_13 = Convolution2D(512, (kernel, kernel), padding="same")(conv_12)
    conv_13 = BatchNormalization()(conv_13)
    conv_13 = Activation("relu")(conv_13)

    pool_5, mask_5 = MaxPoolingWithArgmax2D(pool_size)(conv_13)
    print("Build encoder done..")

    # decoder

    unpool_1 = MaxUnpooling2D(pool_size)([pool_5, mask_5])

    conv_14 = Convolution2D(512, (kernel, kernel), padding="same")(unpool_1)
    conv_14 = BatchNormalization()(conv_14)
    conv_14 = Activation("relu")(conv_14)
    conv_15 = Convolution2D(512, (kernel, kernel), padding="same")(conv_14)
    conv_15 = BatchNormalization()(conv_15)
    conv_15 = Activation("relu")(conv_15)
    conv_16 = Convolution2D(512, (kernel, kernel), padding="same")(conv_15)
    conv_16 = BatchNormalization()(conv_16)
    conv_16 = Activation("relu")(conv_16)

    unpool_2 = MaxUnpooling2D(pool_size)([conv_16, mask_4])

    conv_17 = Convolution2D(512, (kernel, kernel), padding="same")(unpool_2)
    conv_17 = BatchNormalization()(conv_17)
    conv_17 = Activation("relu")(conv_17)
    conv_18 = Convolution2D(512, (kernel, kernel), padding="same")(conv_17)
    conv_18 = BatchNormalization()(conv_18)
    conv_18 = Activation("relu")(conv_18)
    conv_19 = Convolution2D(256, (kernel, kernel), padding="same")(conv_18)
    conv_19 = BatchNormalization()(conv_19)
    conv_19 = Activation("relu")(conv_19)

    unpool_3 = MaxUnpooling2D(pool_size)([conv_19, mask_3])

    conv_20 = Convolution2D(256, (kernel, kernel), padding="same")(unpool_3)
    conv_20 = BatchNormalization()(conv_20)
    conv_20 = Activation("relu")(conv_20)
    conv_21 = Convolution2D(256, (kernel, kernel), padding="same")(conv_20)
    conv_21 = BatchNormalization()(conv_21)
    conv_21 = Activation("relu")(conv_21)
    conv_22 = Convolution2D(128, (kernel, kernel), padding="same")(conv_21)
    conv_22 = BatchNormalization()(conv_22)
    conv_22 = Activation("relu")(conv_22)

    unpool_4 = MaxUnpooling2D(pool_size)([conv_22, mask_2])

    conv_23 = Convolution2D(128, (kernel, kernel), padding="same")(unpool_4)
    conv_23 = BatchNormalization()(conv_23)
    conv_23 = Activation("relu")(conv_23)
    conv_24 = Convolution2D(64, (kernel, kernel), padding="same")(conv_23)
    conv_24 = BatchNormalization()(conv_24)
    conv_24 = Activation("relu")(conv_24)

    unpool_5 = MaxUnpooling2D(pool_size)([conv_24, mask_1])

    conv_25 = Convolution2D(64, (kernel, kernel), padding="same")(unpool_5)
    conv_25 = BatchNormalization()(conv_25)
    conv_25 = Activation("relu",name='25')(conv_25)

    conv_26 = Convolution2D(n_labels, (1, 1), padding="valid")(conv_25)
    conv_26 = BatchNormalization()(conv_26)
    conv_26 = Reshape(
            (input_shape[0] * input_shape[1], n_labels),
            input_shape=(input_shape[0], input_shape[1], n_labels))(conv_26)

    outputs = Activation(output_mode,name='out')(conv_26)
    print("Build decoder done..")

    model = Model(inputs=inputs, outputs=outputs, name="SegNet")

    return model




Using TensorFlow backend.


In [ ]:

#from keras.optimizers import SGD
#INIT_LR = (10**(-3))*1.5
#opt = SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS)

s=MultiSegNetImage([base+'/00-orig.tif'],masks=[base+'/00-mask.tif'],frame=size,overlay=overl,batchSize=batchsize)

model=segnet(
        s.getShape(),
        5,
        kernel=3,
        pool_size=(2, 2))

if training:
    
    model.compile(loss="categorical_crossentropy", optimizer='sgd',
        metrics=["categorical_accuracy"])

    H = model.fit_generator(s,epochs=epochs)
    model.save(base+"/" + modelname)

if not training:
    model.load_weights(base+"/" +modelname)
    from matplotlib import pyplot as plt
    import numpy as np
    from random import shuffle

    plt.figure(figsize = (50,20))
    rs=list(range(len(s)))
    shuffle(rs)
    si=20
    f, axarr = plt.subplots(si,3,figsize=(20,100))

    for i in range(si):
      dta2=model.predict(asarray([s.images[0].getSingleItem(rs[i])]))
      dta2=squeeze(dta2,axis=0)
      dta2=dta2.reshape((size,size,5))

      axarr[i][0].imshow(np.argmax(dta2,axis=2),vmax=4,cmap='hot')

      axarr[i][1].imshow(s.images[0].getSingleItem(rs[i]))
      axarr[i][2].imshow(np.argmax(s.images[0].getMask(rs[i]),axis=2),vmax=4,cmap='hot')

Instructions for updating:
Colocations handled automatically by placer.
Build encoder done..
Build decoder done..
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/3
